In [11]:
from datasets import load_dataset

# Load the JFLEG dataset
dataset = load_dataset("jfleg")
print(dataset)

README.md:   0%|          | 0.00/5.94k [00:00<?, ?B/s]

C:\Users\ankit\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ankit\.cache\huggingface\hub\datasets--jfleg. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


validation-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/141k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 755
    })
    test: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 748
    })
})


In [17]:

# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"grammar correction: {text}" for text in examples["sentence"]]
    targets = [corrections[0] for corrections in examples["corrections"]]  
    return {"input_text": inputs, "target_text": targets}

tokenized_dataset = dataset.map(preprocess_function, batched=True)


model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


def tokenize_function(examples):
    model_inputs = tokenizer(examples["input_text"], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target_text"], max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = tokenized_dataset.map(tokenize_function, batched=True)
print(tokenized_dataset["validation"][0])  # Check an example


Map:   0%|          | 0/755 [00:00<?, ? examples/s]

Map:   0%|          | 0/748 [00:00<?, ? examples/s]

Map:   0%|          | 0/755 [00:00<?, ? examples/s]

Map:   0%|          | 0/748 [00:00<?, ? examples/s]

{'sentence': 'So I think we can not live if old people could not find siences and tecnologies and they did not developped . ', 'corrections': ['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . '], 'input_text': 'grammar correction: So I think we can not live if old people could not find siences and tecnologies and they did not developped . ', 'target_text': 'So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'input_ids': [19519, 11698, 10, 264, 27, 317, 62, 54, 59, 619, 3, 99, 625, 151, 228, 59, 253, 108, 1433, 7, 11, 3, 5822, 29, 4137, 7, 11, 79, 410, 59, 1344, 3138, 3, 5, 1, 0, 0, 